# Notes class

In [196]:
import numpy as np

class Notes:
    def __init__(self, 
                 note_count=100, 
                 notation_count=10, 
                 bits_in_note=8, 
                 bits_in_notation=255,
                 unique_notations=True):
        self.note_count = note_count
        self.bits_in_note = bits_in_note
        self.bits_in_notation = bits_in_notation
        self.notation_count = notation_count
        self.unique_notations=unique_notations
        self._notes = list()  # [note_0_notations, note_1_notations ...]  -> [ [[idx idx ...] ...] ...]
        self.gen_notes()
    
    def __getitem__(self, idx):
        return self._notes[idx]
    
    def __iter__(self):
        for note in self._notes:
            yield note
    
    def __len__(self):
        return len(self._notes)
    
    def notation_as_bits(self, notation) -> np.array:  # [1 0 1 0 0 ...]
        bits = np.zeros(self.bits_in_notation, dtype=np.uint8)
        bits[notation] = 1
        return bits
    
    def note_notation_as_bits(self, note_idx: int, notation_idx: int) -> np.array:  # [1 0 1 0 0 ...]
        true_bits = list(self._notes[note_idx][notation_idx])
        bits = np.zeros(self.bits_in_notation, dtype=np.uint8)
        bits[true_bits] = 1
        return bits

    def gen_notation(self) -> tuple:
        # just true bit indices
        notation = np.random.choice(range(self.bits_in_notation), self.bits_in_note, replace=False)
        return tuple(notation)
        
    def gen_notations(self) -> list:
        notations = set()
        for i in range(self.note_count * self.notation_count):
            while True:
                notation = self.gen_notation()
                if notation not in notations:
                    notations.add(notation)
                    break
        return list(notations)
        
    def gen_notes(self):
        print('start gen notes')
        notations = self.gen_notations()
        self._notes = list()
        for i_note in range(self.note_count):
            note_notations = [notations[i_note * self.notation_count + i] for i in range(self.notation_count)]
            self._notes.append(note_notations)
        print('done')
        
    def phrase(self, notes: list) -> np.array:
        """  notes: [(note_idx, notatiion_idx) ...] -> {bit indices} """
        bits = set()
        for note_idx, notation_idx in notes:
            bits = bits.union(self._notes[note_idx][notation_idx])
        return bits
        

In [191]:
%%time
notes = Notes(note_count=255, 
              notation_count=10, 
              bits_in_note=8, 
              bits_in_notation=255, 
              unique_notations=True)

start gen notes
done
Wall time: 180 ms


In [193]:
notes[0]

[(142, 254, 6, 220, 130, 243, 246, 250),
 (227, 14, 202, 92, 68, 132, 199, 184),
 (136, 124, 79, 48, 195, 140, 146, 73),
 (87, 48, 88, 248, 113, 119, 40, 111),
 (156, 20, 237, 68, 199, 209, 151, 249),
 (62, 150, 191, 102, 72, 56, 242, 16),
 (61, 195, 46, 115, 4, 203, 71, 18),
 (156, 225, 90, 164, 79, 15, 143, 138),
 (26, 119, 95, 16, 99, 126, 185, 35),
 (134, 16, 93, 45, 36, 104, 176, 115)]

In [87]:
from ipythonblocks import BlockGrid

def draw_notation(notation: np.array):
    bit_grid = BlockGrid(len(notation), 1, fill=(17, 41, 129))
    for block in range(bit_grid.width):
        color = bit_grid[0, block]
        if notation[block]:
            bit_grid[0, block] = (244, 195, 173)
    bit_grid.lines_on = False
    bit_grid.show()

def draw_note_notations(notes: Notes, note_idx: int):
    for i in range(len(notes[note_idx])):
        draw_notation(notes.note_notation_as_bits(note_idx, i))

In [194]:
draw_note_notations(notes, 1)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [200]:
phrase_code = [(0, 3), (1, 5), (4, 8), (10, 1), (20, 9)]

for note_idx, notation_idx in phrase_code:
    notation = notes.note_notation_as_bits(note_idx, notation_idx)
    draw_notation(notation)

phrase = notes.phrase(phrase_code)
phrase_bits = notes.notation_as_bits(signal)
print(len(phrase))
print(phrase)
draw_notation(phrase_bits)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


34
{136, 11, 14, 146, 147, 27, 159, 40, 170, 175, 48, 177, 49, 179, 180, 191, 193, 196, 198, 200, 214, 87, 88, 226, 99, 228, 230, 233, 111, 113, 119, 248, 250, 251}


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [199]:
%timeit notes.phrase(phrase_code)

The slowest run took 10.17 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.85 µs per loop


# Numpy drafts

In [113]:
a = np.ndarray(shape=(1,10), dtype=np.bool)
b = np.ndarray(shape=(1,10), dtype=np.bool)
np.append(a, b, axis=0)

array([[False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False]], dtype=bool)

In [167]:

a = np.zeros(shape=(10000,10), dtype=np.bool, order='F')
print(a)
print(a.shape)
b = np.zeros(10, dtype=np.bool)
b[0] = True
print(b)

a[9999] = b
print(a)

%time ((a==b).all(axis=(1))).any()


[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
(10000, 10)
[ True False False False False False False False False False]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [ True False False ..., False False False]]
Wall time: 0 ns


True

In [163]:
rnd_arr = np.random.choice(range(20), 10, replace=False)
tuple(rnd_arr)

(9, 14, 19, 11, 12, 1, 15, 13, 0, 4)

In [136]:
d = {(1, 2), (3, 4), (5, 6), (1, 2)}
d

{(1, 2), (3, 4), (5, 6)}